In [5]:
import csv
import json
import sys
from subprocess import call
import ast
import os
import pandas as pd

In [3]:
#call("pwd")
#os.chdir('./raphtory-mastodon/mastodon')
#for chunck_df in pd.read_csv('znark.us.csv', chunksize=1000):
    # each chunck_df contains a part of the whole CSV
    #chunck_df

### Trying first to parse one of the files as a dataframe. First processing mentions

In [23]:

os.system('ls > lista.txt')
os.system('mkdir grafo')
os.system('mkdir analizar')
l = open('lista.txt', 'r')
a = []
for row in l:
    if ("lista" in row):
        print("")
    else:
        a.append(row.replace("\n", ""))

l.close()

for nombre in a:
    df = pd.read_csv(str(nombre))
    dm = df[df.mentions != '[]' ]
    i = 0
    j= []
    while(i<len(dm['mentions'])):
        j.append(ast.literal_eval(dm['mentions'].iloc[i]))
        dm['mentions'].iloc[i] = j[i]
        i = i + 1
    men = dm['mentions']
    date = dm['created_at']
    user = dm['account.acct']
    us = dm['account.username']

    #Cambiamos el apartado acct por el servidor del usuario que menciona
    i = 0
    j= []
    while(i<len(user)):
        g = str(dm['account.acct'].iloc[i])
        u = str(dm['account.username'].iloc[i])
        j.append(g.replace(u+"@", ""))
        dm['account.acct'].iloc[i] = j[i]
        i = i + 1
    
    #Ponemos los servidores en mentions
    i = 0
    j= []
    while(i<len(user)):
        g = str(dm['mentions'].iloc[i][0]['acct'])
        u = str(dm['mentions'].iloc[i][0]['username'])
        dm['mentions'].iloc[i][0]['acct'] = g.replace(u + "@", "")
        i = i + 1
        
    res = pd.DataFrame()
    res['Date'] = dm['created_at']
    res['UserId'] = dm['account.id']
    res['ServerUser'] = dm['account.acct']
    l = 0
    f = 0
    while(l<len(dm['mentions'])):
        if(len(dm['mentions'].iloc[l])>f):
            f = len(dm['mentions'].iloc[l])
        l = l + 1
    h = 0
    j = 0
    p = f
    m = []
    n = []
    while(j<p):
        while(h<len(men)):
            l1 = len(dm['mentions'].iloc[h])
            if(l1>j):
                m.append(dm['mentions'].iloc[h][j]['id'])
                n.append(dm['mentions'].iloc[h][j]['acct'])
            else:
                m.append(-1)
                n.append(-1)
            h = h + 1
        g = 'UserMentioned' + str(j)
        i = 'ServerMentioned' + str(j)
        h = 0
        res[g] = m
        res[i] = n
        m = []
        n = []
        j = j + 1  
    res.to_csv('./analizar/'+str(nombre))
    #Hacemos los archivos para la carpeta grafo
    enlace = pd.DataFrame()
    nodo = pd.DataFrame()
    y = []
    w = []
    z = 0
    t = 0
    while(z<p):
        s = 'ServerMentioned' + str(z)
        while(t<len(res[s])):
            if (res[s].iloc[t] != -1):
                w.append(res['ServerUser'])
                y.append(res[s])
            t = t + 1
        z = z + 1
    enlace['source'] = pd.concat(w, axis= 0)
    enlace['target'] = pd.concat(y, axis= 0)

    servidores = enlace['source'].unique()
    servidoress = enlace['target'].unique()
    servidores = servidores.tolist()
    num = []
    serv = []
    for t in servidoress:
        if not t in servidores:
            servidores.append(t)
    y = 0
    while(y<len(servidores)):
        num.append(y)
        serv.append(servidores[y])
        y = y + 1
    nodo['Id'] = num
    nodo['Label'] = serv
    enlace.to_csv('./grafo/enlace'+str(nombre))
    nodo.to_csv('./grafo/nodo'+str(nombre))

/var/folders/y4/f611f3kx78q82jwkcq8psnnm0000gn/T/ipykernel_12367/983550832.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm['mentions'].iloc[i] = j[i]
/var/folders/y4/f611f3kx78q82jwkcq8psnnm0000gn/T/ipykernel_12367/983550832.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dm['account.acct'].iloc[i] = j[i]
/var/folders/y4/f611f3kx78q82jwkcq8psnnm0000gn/T/ipykernel_12367/983550832.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-